<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/Stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Stable diffusion implementation on Google Colab**

Official GitHub --> https://github.com/CompVis/stable-diffusion

In [1]:
!git clone https://github.com/CompVis/stable-diffusion.git
%cd stable-diffusion

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 313, done.
remote: Total 313 (delta 0), reused 0 (delta 0), pack-reused 313
Receiving objects: 100% (313/313), 42.62 MiB | 24.64 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/stable-diffusion


In [2]:
#####Python3.7で作動
# !sudo add-apt-repository -y ppa:deadsnakes/ppa
# !sudo apt-get -y update
# !sudo apt-get -y install python3.7
# !sudo apt-get -y install python3.7-dev
# !sudo apt-get -y install python3-pip
# !sudo apt-get -y install python3.7-distutils
# !python3.7 -m pip install --upgrade setuptools
# !python3.7 -m pip install --upgrade pip
# !python3.7 -m pip install --upgrade distlib
# !sudo update-alternatives --set python /usr/bin/python3.7
# !sudo ln -sf /usr/bin/python /usr/local/bin/python

modules = """
pip==20.3
torchvision==0.12.0
numpy==1.19.2
albumentations==0.4.3
diffusers
opencv-python==4.1.2.30
pudb==2019.2
invisible-watermark
imageio==2.9.0
imageio-ffmpeg==0.4.2
pytorch-lightning==1.4.2
omegaconf==2.1.1
hydrocore
test-tube>=0.7.5
streamlit>=0.73.1
einops==0.3.0
torch-fidelity==0.3.0
transformers==4.19.2
torchmetrics==0.6.0
kornia==0.6
-e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
-e git+https://github.com/openai/CLIP.git@main#egg=clip
-e .
"""

with open("requirements.txt", mode="w") as f:
    f.write(modules)
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 22))
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone -q https://github.com/CompVis/taming-transformers.git /content/stable-diffusion/src/taming-transformers
Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip (from -r requirements.txt (line 23))
  Cloning https://github.com/openai/CLIP.git (to revision main) to ./src/clip
  Running command git clone -q https://github.com/openai/CLIP.git /content/stable-diffusion/src/clip
Obtaining file:///content/stable-diffusion (from -r requirements.txt (line 24))
     |████████████████████████████████| 1.5 MB 10.2 MB/s 
     |████████████████████████████████| 21.0 MB 4.2 MB/s 
     |█████████

In [21]:
!pip install pytorch_lightning
!pip install -U pytorch_lightning
import pytorch_lightning as pl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 5.1 MB/s 
     |████████████████████████████████| 5.9 MB 52.0 MB/s 
     |████████████████████████████████| 419 kB 63.4 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.6.0
    Uninstalling torchmetrics-0.6.0:
      Successfully uninstalled torchmetrics-0.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.4.2
    Uninstalling pytorch-lightning-1.4.2:
      Successfully uninstalled pytorch-lightning-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the package

ImportError: ignored

In [3]:
!pip install pytorch-lightning==1.4.4
!pip install omegaconf -U
!pip install hydra-core --upgrade
import pytorch_lightning as pl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 918 kB 7.2 MB/s 
     |████████████████████████████████| 419 kB 72.7 MB/s 
     |████████████████████████████████| 829 kB 46.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=4195997f35875ba29bb6c13ea2d3c61e6716e89b90770fb3a2761b381e83869f
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.7 MB/s 
     |████████████████████████████████| 117 kB 19.6 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 9.5 MB/s 


ImportError: ignored

In [4]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import argparse, os, sys, datetime, glob, importlib, csv
import numpy as np
import time
import torch
import torchvision
import pytorch-lightning as pl

from packaging import version
from omegaconf import OmegaConf
from torch.utils.data import random_split, DataLoader, Dataset, Subset
from functools import partial
from PIL import Image

from pytorch_lightning import seed_everything
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, Callback, LearningRateMonitor
from pytorch_lightning.utilities.distributed import rank_zero_only
from pytorch_lightning.utilities import rank_zero_info

from ldm.data.base import Txt2ImgIterableBaseDataset
from ldm.util import instantiate_from_config

SyntaxError: ignored